In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

In [ ]:
# Starting from Python 3.8 DLL search policy has changed.
# We need to add path to CUDA DLLs explicitly.
import os

if os.name == "nt":
    # Add CUDA_PATH env variable
    cuda_path = os.environ["CUDA_PATH"]
    if cuda_path:
        os.add_dll_directory(os.path.join(cuda_path, "bin"))
    else:
        raise StopExecution

In [ ]:
from PIL import Image
from IPython.display import display
from io import BytesIO

import python_vali as vali
import numpy as np

from ipywidgets import interact
import ipywidgets as widgets
import tempfile

In [ ]:
uploader = widgets.FileUpload(
    accept=".mp4,.avi,.mkv", description="Upload video")
display(uploader)

In [ ]:
def create_temp_file() -> str:
    """
    Create temporary file and write uploader widget output there.
    Temp file will not be auto-deleted.
    """
    with tempfile.NamedTemporaryFile(delete=False) as tmp:
        tmp.write(uploader.value[0]["content"].tobytes())
        return tmp.name

In [ ]:
tmp_file = create_temp_file()

# GPU-accelerated decoder
pyDec = vali.PyDecoder(
    input=tmp_file,
    opts={},
    gpu_id=0)

# GPU-accelerated converter
pyCvt = vali.PySurfaceConverter(
    pyDec.Format,
    vali.PixelFormat.RGB,
    gpu_id=0)

# GPU-accelerated JPEG encoder.
# It's faster to encode Surface on GPU and show JPEG in widget.
pyJpeg = vali.PyNvJpegEncoder(gpu_id=0)
pyJpegEncCtx = pyJpeg.Context(
    compression=100,
    pixel_format=vali.PixelFormat.RGB)

In [ ]:
@interact(seek_frame=widgets.IntSlider(
    min=0, max=pyDec.NumFrames - 1, step=1, value=0))
def decode_show_video_frame(seek_frame: int) -> None:
    """
    Decodes and shows single video frame by number.

    Paramenters
    -----------

    seek_frame: what frame to show
    """
    # Raw decoded Surface
    surf_src = vali.Surface.Make(
        format=pyDec.Format,
        width=pyDec.Width,
        height=pyDec.Height,
        gpu_id=0)

    # Raw Surface, converted to RGB
    surf_dst = vali.Surface.Make(
        format=vali.PixelFormat.RGB,
        width=pyDec.Width,
        height=pyDec.Height,
        gpu_id=0)

    seek_ctx = vali.SeekContext(seek_frame)

    # Decode single Surface
    success, details = pyDec.DecodeSingleSurface(surf_src, seek_ctx)
    if not success:
        print(details)
        raise StopExecution

    # Convert tot RGB
    success, details = pyCvt.Run(surf_src, surf_dst)
    if not success:
        print(details)
        raise StopExecution

    # Compress to JPEG
    buffers, details = pyJpeg.Run(pyJpegEncCtx, [surf_dst])
    if len(buffers) != 1:
        print(details)
        raise StopExecution

    # Display in notebook.
    # The same picture is shown twice for some reason.
    jpeg_bytes = BytesIO(np.ndarray.tobytes(buffers[0]))
    display(Image.open(jpeg_bytes), display_id="decoded_frame")

In [ ]:
os.remove(tmp_file)